### Import Library

In [107]:
import numpy as np
import pandas as pd
import torch
from torch.utils.data import DataLoader, TensorDataset
import torch.nn as nn
import torch.nn.functional as F

torch.set_default_device('cuda')

In [108]:
torch.cuda.get_device_name()

'NVIDIA GeForce RTX 3050 Ti Laptop GPU'

### Load Data

In [109]:
train_data = pd.read_csv('train.csv')
train_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [110]:
test_data = pd.read_csv('test.csv')
test_data.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [111]:
features = ["Pclass", "Sex", "SibSp", "Parch", "Age"]

In [112]:
X = pd.get_dummies(train_data[features]).drop("Sex_female", axis=1).astype(float)
X['Age_missing'] = X['Age'].isna().astype(int)
X['Age'] = X['Age'].fillna(X['Age'].mean())
X_tensor = torch.tensor(X.values, dtype=torch.float32)
X

,Pclass,SibSp,Parch,Age,Sex_male,Age_missing
0,3.0,1.0,0.0,22.000000,1.0,0
1,1.0,1.0,0.0,38.000000,0.0,0
2,3.0,0.0,0.0,26.000000,0.0,0
3,1.0,1.0,0.0,35.000000,0.0,0
4,3.0,0.0,0.0,35.000000,1.0,0
...,...,...,...,...,...,...
886,2.0,0.0,0.0,27.000000,1.0,0
887,1.0,0.0,0.0,19.000000,0.0,0
888,3.0,1.0,2.0,29.699118,0.0,1
889,1.0,0.0,0.0,26.000000,1.0,0


In [113]:
y = train_data['Survived'].values
y_tensor = torch.tensor(y, dtype=torch.long)
y_tensor[:5]

tensor([0, 1, 1, 1, 0], device='cuda:0')

In [114]:
X_test = pd.get_dummies(test_data[features]).drop("Sex_female", axis=1).astype(float)
X_test['Age_missing'] = X_test['Age'].isna().astype(int)
X_test['Age'] = X_test['Age'].fillna(X_test['Age'].mean())
X_test_tensor = torch.tensor(X_test.values, dtype=torch.float32)
X_test

,Pclass,SibSp,Parch,Age,Sex_male,Age_missing
0,3.0,0.0,0.0,34.50000,1.0,0
1,3.0,1.0,0.0,47.00000,0.0,0
2,2.0,0.0,0.0,62.00000,1.0,0
3,3.0,0.0,0.0,27.00000,1.0,0
4,3.0,1.0,1.0,22.00000,0.0,0
...,...,...,...,...,...,...
413,3.0,0.0,0.0,30.27259,1.0,1
414,1.0,0.0,0.0,39.00000,0.0,0
415,3.0,0.0,0.0,38.50000,1.0,0
416,3.0,0.0,0.0,30.27259,1.0,1


In [115]:
train_dataset = TensorDataset(X_tensor, y_tensor)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, generator=torch.Generator(device='cuda'))

In [116]:
test_dataset = TensorDataset(X_test_tensor)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

### Write Model

In [117]:
INPUT_SIZE = X.shape[1]
HIDDEN_SIZE = 3
NUM_CLASSES = 2
INPUT_SIZE

6

In [118]:
class SurvivalClassifier(nn.Module):
    def __init__(self):
        super(SurvivalClassifier, self).__init__()
        self.fc1 = nn.Linear(INPUT_SIZE, HIDDEN_SIZE)
        self.fc2 = nn.Linear(HIDDEN_SIZE, HIDDEN_SIZE)
        self.fc3 = nn.Linear(HIDDEN_SIZE, HIDDEN_SIZE)
        self.fc4 = nn.Linear(HIDDEN_SIZE, HIDDEN_SIZE)
        self.fc5 = nn.Linear(HIDDEN_SIZE, NUM_CLASSES)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [119]:
model = SurvivalClassifier()
model

SurvivalClassifier(
  (fc1): Linear(in_features=6, out_features=3, bias=True)
  (fc2): Linear(in_features=3, out_features=3, bias=True)
  (fc3): Linear(in_features=3, out_features=3, bias=True)
  (fc4): Linear(in_features=3, out_features=3, bias=True)
  (fc5): Linear(in_features=3, out_features=2, bias=True)
)

### Train Model

In [120]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 100

for epoch in range(num_epochs):
    model.train()
    for X_batch, y_batch in train_loader:
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

Epoch [1/100], Loss: 1.0067
Epoch [2/100], Loss: 0.9291
Epoch [3/100], Loss: 0.9218
Epoch [4/100], Loss: 0.7403
Epoch [5/100], Loss: 0.8574
Epoch [6/100], Loss: 0.7165
Epoch [7/100], Loss: 0.7864
Epoch [8/100], Loss: 0.7649
Epoch [9/100], Loss: 0.6171
Epoch [10/100], Loss: 0.6794
Epoch [11/100], Loss: 0.6198
Epoch [12/100], Loss: 0.6669
Epoch [13/100], Loss: 0.5567
Epoch [14/100], Loss: 0.6518
Epoch [15/100], Loss: 0.6392
Epoch [16/100], Loss: 0.7284
Epoch [17/100], Loss: 0.6779
Epoch [18/100], Loss: 0.7114
Epoch [19/100], Loss: 0.6263
Epoch [20/100], Loss: 0.6727
Epoch [21/100], Loss: 0.6221
Epoch [22/100], Loss: 0.7460
Epoch [23/100], Loss: 0.6442
Epoch [24/100], Loss: 0.6939
Epoch [25/100], Loss: 0.6465
Epoch [26/100], Loss: 0.6291
Epoch [27/100], Loss: 0.6554
Epoch [28/100], Loss: 0.6106
Epoch [29/100], Loss: 0.6030
Epoch [30/100], Loss: 0.6706
Epoch [31/100], Loss: 0.5773
Epoch [32/100], Loss: 0.5974
Epoch [33/100], Loss: 0.6078
Epoch [34/100], Loss: 0.6997
Epoch [35/100], Loss: 0

### Get Result

In [121]:
model.eval()

SurvivalClassifier(
  (fc1): Linear(in_features=6, out_features=3, bias=True)
  (fc2): Linear(in_features=3, out_features=3, bias=True)
  (fc3): Linear(in_features=3, out_features=3, bias=True)
  (fc4): Linear(in_features=3, out_features=3, bias=True)
  (fc5): Linear(in_features=3, out_features=2, bias=True)
)

In [122]:
predictions = []

with torch.no_grad():
    for X_batch in test_loader:
        X_batch = X_batch[0]
        outputs = model(X_batch)
        _, predicted = torch.max(outputs.data, 1)
        predictions.extend(predicted.tolist())

predictions_df = pd.DataFrame({'PassengerId': test_data.PassengerId, 'Survived': predictions})
predictions_df

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0


In [123]:
predictions_df.to_csv('submission.csv', index=False)